# PanAnalysis Test Data

## Import required modules

Import the modules required for this solution

In [1]:
import pyodbc
import pandas as pd

## Connect to data source

In [2]:
s_server = '169.254.250.189\SQLEXPRESS'
s_database = 'PanAnalysis'
s_user = 'sa'
s_password = 'Demo'

s_connection = 'DRIVER={SQL Server Native Client 11.0};' 
s_connection += 'SERVER=' + s_server + ';DATABASE=' + s_database +';'
s_connection += 'UID=' + s_user +';PWD=' + s_password

dbconn = pyodbc.connect(s_connection)

print('Database connection made...')

Database connection made...


In [3]:
# Simple database query test
s_tmpsql = 'select top 10 * from zTempPanAnalysis'
dbcursor = dbconn.cursor()
dbcursor.execute(s_tmpsql)
dbrecordset = dbcursor.fetchall()
for record in dbrecordset:
    print('Incident ID :', record.Incident)

print('\nShould have printed 10 6-digit incident IDs.')

Incident ID : 155059
Incident ID : 155060
Incident ID : 155061
Incident ID : 155063
Incident ID : 155065
Incident ID : 155066
Incident ID : 155067
Incident ID : 155068
Incident ID : 155069
Incident ID : 155070

Should have printed 10 6-digit incident IDs.


## Load some data into a dataframe

In [4]:
data_array = []
for record in dbrecordset:
    data_array += [[record.Incident, record.IncidentType, record.CustomerId]]
print('Data array looks like:\n')
print(data_array)

Data array looks like:

[['155059', 'BREAKDOWN', '10023'], ['155060', 'RTF', '12421'], ['155061', 'BREAKDOWN', '12190'], ['155063', 'BREAKDOWN', '11946'], ['155065', 'BREAKDOWN', '12571'], ['155066', 'BREAKDOWN', '10879'], ['155067', 'BREAKDOWN', '11668'], ['155068', 'REPEAT', '13051'], ['155069', 'BREAKDOWN', '11042'], ['155070', 'RTF', '12046']]


In [5]:
df = pd.DataFrame(data_array, columns=['Incident','IncidentType','CustomerId'])
print('Data frame looks like:\n')
print(df)

Data frame looks like:

  Incident IncidentType CustomerId
0   155059    BREAKDOWN      10023
1   155060          RTF      12421
2   155061    BREAKDOWN      12190
3   155063    BREAKDOWN      11946
4   155065    BREAKDOWN      12571
5   155066    BREAKDOWN      10879
6   155067    BREAKDOWN      11668
7   155068       REPEAT      13051
8   155069    BREAKDOWN      11042
9   155070          RTF      12046


In [6]:
# Get the columns from the recordset
s_cols = [col[0] for col in dbcursor.description]
print('Columns list:\n', s_cols)

Columns list:
 ['ID', 'Incident', 'IncidentType', 'CustomerId', 'LedgerCode', 'BusinessType', 'SiteId', 'MachinesOnSite', 'SoleMachine', 'Manufacturer', 'SerialNo', 'ProductId', 'DeviceType', 'Colour', 'AncilliariesCount', 'FirstUsed', 'FirstUsedDays', 'Installed', 'InstalledDays', 'PreviousBreakdowns', 'LastBreakCall', 'LastBreakCallDays', 'LastOtherCall', 'LastOtherCallDays', 'LastOtherCallType', 'InitialMeterValueBlack', 'InitialMeterValueColour', 'InitialMeterReadingDate', 'LastMeterValueBlack', 'LastMeterValueColour', 'LastMeterReadingDate', 'BlackClickPerDay', 'ColourClickPerDay', 'TotalClickPerDay', 'CreatedBy', 'CreatedDateTime', 'CreatedTime', 'CreatedDay', 'AttendDateTime', 'AttendDay', 'AttendHour', 'CallMinutes', 'MinutesToAttend', 'PostCodeArea', 'FirstEngineer']


In [25]:
# This should fetch all of the rows and give the sensible column names
s_tmpsql2 = 'select top 10 * from zTempPanAnalysis'
df2 = pd.read_sql_query(s_tmpsql2, dbconn)

In [26]:
print(df2)


     ID Incident IncidentType CustomerId LedgerCode  \
0  1796   155059    BREAKDOWN      10023       M440   
1  1797   155060          RTF      12421       T278   
2  1798   155061    BREAKDOWN      12190      GP114   
3  1799   155063    BREAKDOWN      11946       C614   
4  1800   155065    BREAKDOWN      12571       H327   
5  1801   155066    BREAKDOWN      10879      MW016   
6  1802   155067    BREAKDOWN      11668       N219   
7  1803   155068       REPEAT      13051       E226   
8  1804   155069    BREAKDOWN      11042      SH067   
9  1805   155070          RTF      12046      MR002   

                           BusinessType SiteId  MachinesOnSite  SoleMachine  \
0                 Manufacturing - Other  21936               7        False   
1     Education - Secondary/High school  23678              16        False   
2                   Housing Association  24064              14        False   
3     Education - Secondary/High school  20076              14        False   

In [27]:
# Can now mess about with the dataframe
y = df['IncidentType']
print('Extracted:\n',y)

Extracted:
 0    BREAKDOWN
1          RTF
2    BREAKDOWN
3    BREAKDOWN
4    BREAKDOWN
5    BREAKDOWN
6    BREAKDOWN
7       REPEAT
8    BREAKDOWN
9          RTF
Name: IncidentType, dtype: object


In [29]:
data = df2
del data['IncidentType']
print('Feature data:\n',data)

Feature data:
      ID Incident CustomerId LedgerCode                          BusinessType  \
0  1796   155059      10023       M440                 Manufacturing - Other   
1  1797   155060      12421       T278     Education - Secondary/High school   
2  1798   155061      12190      GP114                   Housing Association   
3  1799   155063      11946       C614     Education - Secondary/High school   
4  1800   155065      12571       H327     Education - Secondary/High school   
5  1801   155066      10879      MW016     Education - Secondary/High school   
6  1802   155067      11668       N219     Education - Secondary/High school   
7  1803   155068      13051       E226  Manufacturing - Chemicals/Fuels/Oils   
8  1804   155069      11042      SH067     Professional - Architect/Surveyor   
9  1805   155070      12046      MR002     Education - Secondary/High school   

  SiteId  MachinesOnSite  SoleMachine Manufacturer     SerialNo      ...       \
0  21936               

## Need to do some management of the data - e.g. categoricals to IDs etc

1. Convert categorical data to IDs - e.g. engineer names, postcodes - consider if one-hot encoding is better here
2. Remove fields we don't need - e.g. redundant datatime values, incident IDs, serial numbers


In [31]:
# Now build a decision tree classifier on this data
from sklearn.tree import DecisionTreeClassifier
dtree=DecisionTreeClassifier()
del data['FirstEngineer']
dtree.fit(data,y)

ValueError: could not convert string to float: 'OL9'